In [1]:
import spacy
import contextualSpellCheck

nlp = spacy.load('en_core_web_sm')


In [3]:
import pandas as pd 
        
# Read csv file  any
table = pd.read_csv("../voxya/voxya_data_v1.csv") 
table_n = table.shape[0]

In [58]:
table

,Unnamed: 0,url,victim,scammer,date,desc,label
0,0,https://voxya.com/view-complaint/fraudism/117445,https://voxya.com/user-complaint/123603,https://voxya.com/company/resume-to-fill-com-c...,"Apr 01, 2021",Mera naam Shreya hai mene freelance me I'd diy...,OPEN
1,1,https://voxya.com/view-complaint/non-payment-o...,https://voxya.com/user-complaint/123400,https://voxya.com/company/kiran-motors-mumbai-...,"Mar 30, 2021",This is regarding my pending payment from Kira...,OPEN
2,2,https://voxya.com/view-complaint/salary-not-pa...,https://voxya.com/user-complaint/123304,https://voxya.com/company/kastech-group-of-com...,"Mar 30, 2021","I Am Surender Nawandar, Residing In Hyderabad,...",OPEN
3,3,https://voxya.com/view-complaint/fake-job/116536,https://voxya.com/user-complaint/122676,https://voxya.com/company/naukri.com-fast-forw...,"Mar 24, 2021",Fake job call through fake email id hr@naukri-...,OPEN
4,4,https://voxya.com/view-complaint/fraud-via-cal...,https://voxya.com/user-complaint/122589,https://voxya.com/company/naukri.com-complaint...,"Mar 23, 2021",I got a call from naukri.com \r\nThey said tha...,OPEN
...,...,...,...,...,...,...,...
563,563,https://voxya.com/view-complaint/refund-not-do...,https://voxya.com/user-complaint/24412,https://voxya.com/company/appolo-hospital-comp...,"Apr 11, 2019",I paid rupees ***** to Surbhi Agrawal for job ...,CLOSED
564,564,https://voxya.com/view-complaint/fraud-done-by...,https://voxya.com/user-complaint/24378,https://voxya.com/company/afflatus-creative-so...,"Apr 11, 2019",Owner of the company Mr Avinash Anshu stolen m...,CLOSED
565,565,https://voxya.com/view-complaint/not-paying-my...,https://voxya.com/user-complaint/24379,https://voxya.com/company/rich-live-trade-comp...,"Apr 11, 2019","I have been working in a company called "" Rich...",CLOSED
566,566,https://voxya.com/view-complaint/fake-data-ent...,https://voxya.com/user-complaint/24354,https://voxya.com/company/adsmacro.com-complai...,"Apr 11, 2019",Please don't trust the site onlinedataentryjob...,CLOSED


In [5]:
def rule3(text):
    
    doc = nlp(text)
    
    sent = []
    
    for token in doc:
        
        # if the token is a verb
        if (token.pos_=='VERB'):
            
            phrase =''
            
            # only extract noun or pronoun subjects
            for sub_tok in token.rights:
                
                if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    # save the root of the verb in phrase
                    
                    phrase += token.lemma_ 
                    # add subject to the phrase
                    phrase += ' '+sub_tok.text

                    # check for noun or pronoun direct objects
                    sent.append(phrase)
            
    return sent

In [6]:
def checkifoneof (test, stri):
    for i in test :
        if stri.find(i)>-1:
            return True
    return False 
import re


In [75]:
list_= []
list_via = []
dict_ ={}
count = 0
count1 = 0
count3 = 0
count2 = 0
countz = 0
truthTable = [["prof. networks","messaging apps","call","email","text","corona","interviewed"]]
for i in range(table_n):
    currentRow = []
    if i%100 == 0 :
        print(i,count,count1,count2,count3,countz)
    desc = str(table["desc"][i]).lower()
    
    #online platforms
    if checkifoneof(["timesjobs","linkedin","linked in","linked-in"," times jobs "," olx ","naukri","glassdoor","naukari"," shine ","shine.com","quikr"],desc):
        count1+=1
        currentRow.append(True)
    else :
        currentRow.append(False)
        
        
    if checkifoneof(["telegram","whatsapp","whatsup","whats app","messenger","hangout"],desc):
        count2+=1
        currentRow.append(True)
    else :
        currentRow.append(False)
        
    phrases = rule3(str(desc).lower())
    via = desc.find('via')
    thr = desc.find('through')
    thru = desc.find(" thru ")
    if (via>-1 or thr>-1 or thru>-1):
        if (via > -1  and checkifoneof(["call","phone"],desc[via:via+20])):
            countz+=1
            currentRow.append(True)
        elif (thr > -1  and checkifoneof(["call","phone"],desc[thr:thr+20])):
            countz+=1
            currentRow.append(True)
        elif (thru > -1  and checkifoneof(["call","phone"],desc[thru:thru+20])):
            countz+=1
            currentRow.append(True)
        else :
            currentRow.append(False)
    else :      
        currentRow.append(False)
        for sent in phrases:
            v, n2 = sent.split()[0], sent.split()[1:]
            if v == "receive" or v == "get":
                if checkifoneof(["call","phonecall"]," ".join(n2)):
                    countz+=1
                    currentRow[-1] = True
                    break
        
    if (checkifoneof(["email","e-mail","e mail"],desc)):
        count+=1
        currentRow.append(True)
    else :
        currentRow.append(False)

    if (checkifoneof(["sms","messag"],desc)):
        count3+=1
        currentRow.append(True)
    else :
        currentRow.append(False)        

    if (checkifoneof(["covid","corona"," pandemic "],desc)):
        currentRow.append(True)
    else :
        currentRow.append(False)    
        
    if (checkifoneof(["interview"],desc)):
        currentRow.append(True)
    else :
        currentRow.append(False)    
        
    truthTable.append(currentRow)
    


0 0 0 0 0 0
100 26 21 7 9 28
200 60 43 15 31 43
300 93 60 27 44 58
400 117 68 34 64 82
500 138 87 38 75 97


In [73]:
#covid related
ans = 0
for z in range(len(truthTable)):
    i = truthTable[z]
    if i[-1] == True:
        ans+=1
ans

114

In [74]:
#unidentified methods of engagement
ans = 0
for z in (truthTable):
    for x in z[0:-2]:
        if x == True:
            ans+=1
            break
ans = len(truthTable)-ans
ans

261

In [77]:
truthTable

[['prof. networks',
  'messaging apps',
  'call',
  'email',
  'text',
  'corona',
  'interviewed'],
 [False, False, False, False, True, False, False],
 [False, False, False, False, False, False, False],
 [False, False, False, False, False, False, False],
 [True, False, False, True, False, False, False],
 [True, False, True, False, False, False, False],
 [False, False, False, False, True, False, True],
 [False, True, False, False, True, False, False],
 [True, False, True, False, False, False, True],
 [True, False, False, False, False, False, True],
 [False, False, False, False, False, False, False],
 [True, False, False, False, False, False, True],
 [False, False, False, True, False, False, True],
 [False, False, False, False, False, False, True],
 [False, False, False, False, False, False, True],
 [False, False, False, False, False, False, False],
 [False, False, False, False, False, False, False],
 [False, True, False, False, False, True, False],
 [False, False, True, False, False, F

In [50]:
print("email "+str(count), "linkedin "+ str(count1),"telegram " +str(count2)," text : "+str(count3),"Call" + str(countz))

email 154 linkedin 62 telegram 42  text : 91 Call109
